### **1. Setup**

In [1]:
import os, requests, sys, json
import numpy as np
import pandas as pd
from time import sleep

In [2]:
# Data import
# the product from MemBrain_2nd Part
# The list of proteins in selected organisms
# With subcellular localization info from Uniprot
path = './IntermediateProducts/Result_Selected_Organisms_SubCellLoc.csv'
df = pd.read_csv(path)

In [3]:
df.shape
# df.head()

(2992, 7)

In [34]:
df[df['Protein_name'].str.contains('209')]

,Uniprot_ID,Organism,Protein_name,AH_or_Not,AA_sequence,Prediction,Subcellular_location
468,Q9FVQ5,Arabidopsis thaliana (Mouse-ear cress),CSC1-like protein At1g32090,AH,MATLQDIGVSALINLFGAFLFLIAFAVLRIQPINDRVYFPKWYLTG...,0000000000000000000000000000000000000000000000...,"Golgi apparatus membrane, Cell membrane"
1974,Q5JX69,Homo sapiens (Human),Protein FAM209B,AH,MWTLKSSLVLLLCLTCSYAFMFSSLRQKTSEPQGKVPCGEHFRIRQ...,0000000000000000000000000000000000000000000000...,Membrane
2100,Q68FR5,Rattus norvegicus (Rat),Transmembrane protein 209,AH,MMQGEVSPSPSLIDRTIRMRKETESRKVVLAWGLLNVSMAGMIYTE...,0000000000111111111111111111000000000000000000...,Membrane


### 2. Merge with proteome data

### 2-1. Schirmer 2003

In [4]:
df_shirmer2003 = pd.read_csv('../Nuclear_proteome/Output/Schirmer2003/Output.csv')

In [5]:
df_shirmer2003.shape
# df_shirmer2003.head()

(56, 6)

#### Inner merge to find nly three matched

In [6]:
df_merged_inner = df.merge(df_shirmer2003, how='inner', left_on='Uniprot_ID', right_on='Entry')

In [7]:
df_merged_inner.shape

(3, 13)

#### Outer merge to assign sub-cell-loc info from Schirmer data

In [8]:
df_merged_outer = df.merge(df_shirmer2003, how='outer', left_on='Uniprot_ID', right_on='Entry')

In [9]:
df_merged_outer.shape

(3045, 13)

In [56]:
df_merged_outer.head()

,Uniprot_ID,Organism_x,Protein_name,AH_or_Not,AA_sequence,Prediction,Subcellular_location,NCBI ID,Entry,Entry name,Protein names,Gene names,Organism_y
0,Q8N4K4,Homo sapiens (Human),Reprimo-like protein,Non-AH,MNATFLNHSGLEEVDGVGGGAGAALGNRTHGLGTWLGCCPGGAPLA...,0000000000000000000000000000000000000000000000...,Membrane,NaN,NaN,NaN,NaN,NaN,NaN
1,Q8N4S7,Homo sapiens (Human),Progestin and adipoQ receptor family member 4,Non-AH,MAFLAGPRLLDWASSPPHLQFNKFVLTGYRPASSGSGCLRSLFYLH...,0000000000000000000000000000000000000000000000...,Membrane,NaN,NaN,NaN,NaN,NaN,NaN
2,Q8N5G0,Homo sapiens (Human),Small integral membrane protein 20,Non-AH,MSRNLRTALIFGGFISLIGAAFYPIYFRPLMRLEEYKKEQAINRAG...,0000000000000000000000000000000000000000000000...,Mitochondrion inner membrane,NaN,NaN,NaN,NaN,NaN,NaN
3,Q8N614,Homo sapiens (Human),Transmembrane protein 156,AH,MTKTALLKLFVAIVITFILILPEYFKTPKERTLELSCLEVCLQSNF...,0011111100000000000000000000000000000000000000...,Membrane,NaN,NaN,NaN,NaN,NaN,NaN
4,Q8N7C4,Homo sapiens (Human),Transmembrane protein 217,Non-AH,MKQQQWCGMTAKMGTVLSGVFTIMAVDMYLIFEQKHLGNGSCTEIT...,0000000000000000000000000000000000000000000000...,Membrane,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# drop where Uniprot_ID is Nan
df_merged_outer = df_merged_outer.dropna(subset=['Uniprot_ID'])

# drop unnecessary columns
df_merged_outer = df_merged_outer.drop([ 'NCBI ID', 'Entry name',
       'Protein names', 'Gene names', 'Organism_y'], axis=1)

# Assign a value 'Nucleus membrane' to a new column 'Subcellular_location_Schirmer2003' where 'Entry' is present
df_merged_outer.loc[pd.notna(df_merged_outer['Entry']), 'Subcellular_location_Schirmer2003'] = 'Nucleus membrane'

In [12]:
df_merged_outer.info()
# df_merged_outer.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2992 entries, 0 to 2991
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Uniprot_ID                         2992 non-null   object
 1   Organism_x                         2992 non-null   object
 2   Protein_name                       2992 non-null   object
 3   AH_or_Not                          2992 non-null   object
 4   AA_sequence                        2992 non-null   object
 5   Prediction                         2992 non-null   object
 6   Subcellular_location               2992 non-null   object
 7   Entry                              3 non-null      object
 8   Subcellular_location_Schirmer2003  3 non-null      object
dtypes: object(9)
memory usage: 233.8+ KB


In [14]:
# drop an unnecessary column, 'Entry'
df_merged_outer = df_merged_outer.drop(['Entry'], axis=1)

### 2-2. Korfali 2012

In [81]:
df_korfali2012 = pd.read_csv('../Nuclear_proteome/Output/Korfali2012_HsMmRn.csv')

In [82]:
df_korfali2012.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            119 non-null    int64 
 1   tissue                119 non-null    object
 2   gene name             119 non-null    object
 3   alternate names       119 non-null    object
 4   accession numbers     119 non-null    object
 5   NE:MM ratio by dNSAF  119 non-null    object
 6   reference             119 non-null    object
 7   Uniprot_id_Hs         119 non-null    object
 8   Uniprot_id_Rn         119 non-null    object
 9   Uniprot_id_Mm         119 non-null    object
dtypes: int64(1), object(9)
memory usage: 9.4+ KB


In [83]:
df_korfali2012.head()

,Unnamed: 0,tissue,gene name,alternate names,accession numbers,NE:MM ratio by dNSAF,reference,Uniprot_id_Hs,Uniprot_id_Rn,Uniprot_id_Mm
0,0,liver enriched,TMEM53,"NET4, transmembrane protein 53",ref|NP_081113.1|,2.57,"This study and Schirmer, E.C., et al. (2003). ...",Q6P2H8,D3ZPB8,Q9D0Z3
1,1,liver enriched,TMEM120A,"NET29, transmembrane protein induced by tumor ...",ref|NP_766129.1|,INF,"This study and Malik, P., et al. (2010) Cell M...",Q9BXJ8,Q5HZE2,Q8C1E7
2,2,liver enriched,SCARA5,"NET33, PREDICTED: similar to protease, serine,...",gi|109502608|ref|XP_001066668.1|,0.1,"This study and Malik, P., et al. (2010) Cell M...",Q6ZMJ2,D4A213,Q8K299
3,3,liver enriched,TMEM74,"NET36, PREDICTED: hypothetical protein [Rattus...",ref|XP_001063530.1|,3.36,"This study and Malik, P., et al. (2010) Cell M...",Q96NL1,D3ZR33,Q8BQU7
4,4,liver enriched,PPAPDC3,"NET39, phosphatidic acid phosphatase type 2 do...",gi|59891419|ref|NP_001012349.1|;gi|34147436|re...,4.42,"This study and Schirmer, E.C., et al. (2003). ...",Q8NBV4,Q5FVJ3,Q91WB2


#### Outer merge to find the match of Uniprot ID while keeping unmatched rows

#### Human

In [84]:
df_merged_korfali2012_Hs = df.merge(df_korfali2012, how='outer', left_on='Uniprot_ID', right_on='Uniprot_id_Hs')

In [85]:
df_merged_korfali2012_Hs = df_merged_korfali2012_Hs[['Uniprot_ID', 'Organism', 'Protein_name', 'AH_or_Not', 'AA_sequence',
       'Prediction', 'Subcellular_location', 'Uniprot_id_Hs']]

#### Mouse

In [86]:
df_merged_korfali2012_Hs_Mm = df_merged_korfali2012_Hs.merge(df_korfali2012, how='outer', left_on='Uniprot_ID', right_on='Uniprot_id_Mm')

In [87]:
df_merged_korfali2012_Hs_Mm.columns

Index(['Uniprot_ID', 'Organism', 'Protein_name', 'AH_or_Not', 'AA_sequence',
       'Prediction', 'Subcellular_location', 'Uniprot_id_Hs_x', 'Unnamed: 0',
       'tissue', 'gene name', 'alternate names', 'accession numbers',
       'NE:MM ratio by dNSAF', 'reference', 'Uniprot_id_Hs_y', 'Uniprot_id_Rn',
       'Uniprot_id_Mm'],
      dtype='object')

In [88]:
df_merged_korfali2012_Hs_Mm = df_merged_korfali2012_Hs_Mm.rename(columns={'Uniprot_id_Hs_x': 'Uniprot_id_Hs'})

In [89]:
df_merged_korfali2012_Hs_Mm = df_merged_korfali2012_Hs_Mm[['Uniprot_ID', 'Organism', 'Protein_name', 'AH_or_Not', 'AA_sequence',
       'Prediction', 'Subcellular_location', 'Uniprot_id_Hs','Uniprot_id_Mm']]

#### Rat

In [91]:
df_merged_korfali2012_Hs_Mm_Rn = df_merged_korfali2012_Hs_Mm.merge(df_korfali2012, how='outer', left_on='Uniprot_ID', right_on='Uniprot_id_Rn')

In [92]:
df_merged_korfali2012_Hs_Mm_Rn.columns

Index(['Uniprot_ID', 'Organism', 'Protein_name', 'AH_or_Not', 'AA_sequence',
       'Prediction', 'Subcellular_location', 'Uniprot_id_Hs_x',
       'Uniprot_id_Mm_x', 'Unnamed: 0', 'tissue', 'gene name',
       'alternate names', 'accession numbers', 'NE:MM ratio by dNSAF',
       'reference', 'Uniprot_id_Hs_y', 'Uniprot_id_Rn', 'Uniprot_id_Mm_y'],
      dtype='object')

In [93]:
df_merged_korfali2012_Hs_Mm_Rn = df_merged_korfali2012_Hs_Mm_Rn.rename(columns={'Uniprot_id_Hs_x': 'Uniprot_id_Hs', 'Uniprot_id_Mm_x': 'Uniprot_id_Mm'})

In [94]:
df_merged_korfali2012_Hs_Mm_Rn = df_merged_korfali2012_Hs_Mm_Rn[['Uniprot_ID', 'Organism', 'Protein_name', 'AH_or_Not', 'AA_sequence',
       'Prediction', 'Subcellular_location', 'Uniprot_id_Hs','Uniprot_id_Mm', 'Uniprot_id_Rn']]

#### Remove unmatched rows

In [95]:
df_merged_korfali2012_Hs_Mm_Rn = df_merged_korfali2012_Hs_Mm_Rn.dropna(subset=['Uniprot_ID'])

#### If Uniprot ID from Korfali is present, append Subcellular location "Nucleus envelope"

In [97]:
df_merged_korfali2012_Hs_Mm_Rn.loc[(~_df['Uniprot_id_Hs_x'].isnull())|(~_df['Uniprot_id_Mm_x'].isnull())|(~_df['Uniprot_id_Rn'].isnull()), 'Subcellular_location_Korfali2012'] = 'Nucleus envelope'

In [99]:
df_merged_korfali2012_Hs_Mm_Rn[df_merged_korfali2012_Hs_Mm_Rn['Subcellular_location_Korfali2012'] == 'Nucleus envelope']

,Uniprot_ID,Organism,Protein_name,AH_or_Not,AA_sequence,Prediction,Subcellular_location,Uniprot_id_Hs,Uniprot_id_Mm,Uniprot_id_Rn,Subcellular_location_Korfali2012
169,Q91YX5,Mus musculus (Mouse),Acyl-CoA:lysophosphatidylglycerol acyltransfer...,AH,MAVTVEEAPWLGWIVAKALMRFAFMVANNLVAIPSYICYVIILQPL...,0000000111111111111100000000000000000000000000...,Endoplasmic reticulum membrane,NaN,Q91YX5,NaN,Nucleus envelope
330,Q9BUB7,Homo sapiens (Human),"Transmembrane protein 70, mitochondrial",AH,MLFLALGSPWAVELPLCGRRTALCAAAALRGPRASVSRASSSSGPS...,0000000000000000000000000000000000000000000000...,Mitochondrion inner membrane,Q9BUB7,NaN,NaN,Nucleus envelope
381,Q9D666,Mus musculus (Mouse),SUN domain-containing protein 1,AH,MDFSRLHTYTPPQCVPENTGYTYALSSSYSSDALDFETEHKLEPVF...,0000000000000000000000000000000000000000000000...,Nucleus inner membrane,NaN,Q9D666,NaN,Nucleus envelope
493,Q9H1E5,Homo sapiens (Human),Thioredoxin-related transmembrane protein 4,AH,MAGGRCGPQLTALLAAWIAAVAATAGPEEAALPPEQSRVQPMTASN...,0000000000000000000000000000000000000000000000...,Nucleus inner membrane,Q9H1E5,NaN,NaN,Nucleus envelope
499,Q9H330,Homo sapiens (Human),Transmembrane protein 245,AH,MADGGGPKDAPSLRSSPGPAPRVPRAVGPSGGGGETPRTAALALRF...,0000000000000000000000000000000000000000000000...,Membrane,Q9H330,NaN,NaN,Nucleus envelope
675,Q9NX78,Homo sapiens (Human),Transmembrane protein 260,AH,MSPHGDGRGQAQGRAVRVGLRRSGGIRGGVAVFAAVAAVFTFTLPP...,0000000000000000000000000000000000000000000000...,Membrane,Q9NX78,NaN,NaN,Nucleus envelope
676,Q9NXE4,Homo sapiens (Human),Sphingomyelin phosphodiesterase 4,Non-AH,MTTFGAVAEWRLPSLRRATLWIPQWFAKKAIFNSPLEAAMAFPHLQ...,0000000000000000000000000000000000000000000000...,"Endoplasmic reticulum membrane, Golgi apparatu...",Q9NXE4,NaN,NaN,Nucleus envelope
899,Q9WU40,Mus musculus (Mouse),Inner nuclear membrane protein Man1,AH,MAAATAAAAPQQLSDEELFSQLRRYGLSPGPVTESTRPVYLKKLKK...,0000000000000000000000000000000000000000000000...,Nucleus inner membrane,NaN,Q9WU40,NaN,Nucleus envelope
1030,O75425,Homo sapiens (Human),Motile sperm domain-containing protein 3,Non-AH,MRRGAPQDQELVGPGPPGRGSRGAPPPLGPVVPVLVFPPDLVFRAD...,0000000000000000000000000000000000000000000000...,Membrane,O75425,NaN,NaN,Nucleus envelope
1041,O76024,Homo sapiens (Human),Wolframin,AH,MDSNTAPLGPSCPQPPPAPQPQARSRLNATASLEQERSERPRAPGP...,0000000000000000000000000000000000000000000000...,"Endoplasmic reticulum membrane, Cytoplasmic ve...",O76024,NaN,NaN,Nucleus envelope


In [100]:
df_merged_korfali2012_Hs_Mm_Rn = df_merged_korfali2012_Hs_Mm_Rn[['Uniprot_ID','Subcellular_location_Korfali2012']]

In [101]:
df_update = df.merge(df_merged_korfali2012_Hs_Mm_Rn, how='outer', on='Uniprot_ID')

In [104]:
df_update = df_update.dropna(subset=['Uniprot_ID'])

In [105]:
df_update.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2992 entries, 0 to 2991
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Uniprot_ID                        2992 non-null   object
 1   Organism                          2992 non-null   object
 2   Protein_name                      2992 non-null   object
 3   AH_or_Not                         2992 non-null   object
 4   AA_sequence                       2992 non-null   object
 5   Prediction                        2992 non-null   object
 6   Subcellular_location              2992 non-null   object
 7   Subcellular_location_Korfali2012  23 non-null     object
dtypes: object(8)
memory usage: 210.4+ KB


In [106]:
df_update.to_csv('./IntermediateProducts/Result_Selected_Organisms_SubCellLoc_Korfali2012.csv')

### 2-3. HPA

In [13]:
df_hpa = pd.read_csv('../Nuclear_proteome/Output/HPA_NE-NP-ER_Hs.csv')

In [14]:
df_hpa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7601 entries, 0 to 7600
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            7601 non-null   int64 
 1   Gene_name             7601 non-null   object
 2   Subcellular_loc_HPA   7601 non-null   object
 3   EvidenceLevel_HPA     7601 non-null   object
 4   Subcellular_location  7601 non-null   object
 5   Uniprot_id            7601 non-null   object
dtypes: int64(1), object(5)
memory usage: 356.4+ KB


In [36]:
df_hpa[df_hpa['Gene_name'].str.contains('209')]

,Unnamed: 0,Gene_name,Subcellular_loc_HPA,EvidenceLevel_HPA,Subcellular_location,Uniprot_id
175,176,TMEM209,"['Nuclear membrane', 'Nuclear speckles', 'Vesi...",Approved,Nuclear_membrane,Q96SK2
5958,6065,TMEM209,"['Nuclear membrane', 'Nuclear speckles', 'Vesi...",Approved,Nucleoplasm,Q96SK2
7500,332,AC020907.6,['Nucleoplasm'],Approved,Nucleoplasm,Not_found


In [29]:
df_hpa_dup = df_hpa.drop_duplicates(subset=['Gene_name'])
df_hpa_dup

,Unnamed: 0,Gene_name,Subcellular_loc_HPA,EvidenceLevel_HPA,Subcellular_location,Uniprot_id
0,0,TPR,['Nuclear membrane'],Enhanced,Nuclear_membrane,P12270
1,1,XPO1,"['Nucleoplasm', 'Nuclear membrane']",Enhanced,Nuclear_membrane,O14980
2,2,NUP50,"['Nucleoplasm', 'Nuclear membrane']",Enhanced,Nuclear_membrane,Q9UKX7
3,3,RANGAP1,['Nuclear membrane'],Enhanced,Nuclear_membrane,P46060
4,4,EMD,['Nuclear membrane'],Enhanced,Nuclear_membrane,P50402
...,...,...,...,...,...,...
7596,7095,AP001781.2,['Endoplasmic reticulum'],Approved,ER,Not_found
7597,7236,GIMAP1-GIMAP5,['Endoplasmic reticulum'],Supported,ER,A0A087WTJ2
7598,7410,RPL17-C18orf32,"['Endoplasmic reticulum', 'Cytosol']",Supported,ER,A0A0A6YYL6
7599,7426,RPL36A-HNRNPH2,"['Endoplasmic reticulum', 'Cytosol']",Approved,ER,H0Y3V9


In [15]:
df_merged_hpa_inner = df.merge(df_hpa, how='inner', left_on='Uniprot_ID', right_on='Uniprot_id')

In [16]:
df_merged_hpa_inner.shape

(211, 13)

In [31]:
df_hpa[df_hpa['Gene_name'] == 'SCAI']

,Unnamed: 0,Gene_name,Subcellular_loc_HPA,EvidenceLevel_HPA,Subcellular_location,Uniprot_id
84,85,SCAI,"['Nucleoplasm', 'Nuclear membrane']",Supported,Nuclear_membrane,Q8N9R8
5074,5176,SCAI,"['Nucleoplasm', 'Nuclear membrane']",Supported,Nucleoplasm,Q8N9R8


In [32]:
df_merged_hpa_inner_NewNE = df_merged_hpa_inner[(~df_merged_hpa_inner['Subcellular_location_x'].str.contains('Nucleus'))
                                               & (df_merged_hpa_inner['Subcellular_loc_HPA'].str.contains('Nuclear membrane'))]

In [33]:
df_merged_hpa_inner_NewNE

,Uniprot_ID,Organism,Protein_name,AH_or_Not,AA_sequence,Prediction,Subcellular_location_x,Unnamed: 0,Gene_name,Subcellular_loc_HPA,EvidenceLevel_HPA,Subcellular_location_y,Uniprot_id
75,O75631,Homo sapiens (Human),Uroplakin-3a,Non-AH,MPPLWALLALGCLRFGSAVNLQPQLASVTFATNNPTLTTVALEKPL...,0000000000000000000000000000000000000000000000...,Endoplasmic reticulum membrane,268,UPK3A,['Nuclear membrane'],Uncertain,Nuclear_membrane,O75631
76,O75631,Homo sapiens (Human),Uroplakin-3a,Non-AH,MPPLWALLALGCLRFGSAVNLQPQLASVTFATNNPTLTTVALEKPL...,0000000000000000000000000000000000000000000000...,Endoplasmic reticulum membrane,6383,UPK3A,['Nuclear membrane'],Uncertain,Nucleoplasm,O75631
90,P0DJ93,Homo sapiens (Human),Small integral membrane protein 13,AH,MWHSVGLTLLVFVATLLIVLLLMVCGWYFVWHLFLSKFKFLRELVG...,0000000000000000000000000000000000011111111110...,Membrane,260,SMIM13,"['Nucleoplasm', 'Nuclear membrane', 'Golgi app...",Approved,Nuclear_membrane,P0DJ93
91,P0DJ93,Homo sapiens (Human),Small integral membrane protein 13,AH,MWHSVGLTLLVFVATLLIVLLLMVCGWYFVWHLFLSKFKFLRELVG...,0000000000000000000000000000000000011111111110...,Membrane,5487,SMIM13,"['Nucleoplasm', 'Nuclear membrane', 'Golgi app...",Approved,Nucleoplasm,P0DJ93
123,Q14714,Homo sapiens (Human),Sarcospan,Non-AH,MGKNKQPRGQQRQGGPPAADAAGPDDMEPKKGTGAPKECGEEEPRT...,0000000000000000000000000000000000000000000000...,"Cell membrane, Cell membrane, sarcolemma, Cell...",149,SSPN,['Nuclear membrane'],Approved,Nuclear_membrane,Q14714
124,Q14714,Homo sapiens (Human),Sarcospan,Non-AH,MGKNKQPRGQQRQGGPPAADAAGPDDMEPKKGTGAPKECGEEEPRT...,0000000000000000000000000000000000000000000000...,"Cell membrane, Cell membrane, sarcolemma, Cell...",5684,SSPN,['Nuclear membrane'],Approved,Nucleoplasm,Q14714
155,Q6P9G4,Homo sapiens (Human),Transmembrane protein 154,Non-AH,MQAPRAALVFALVIALVPVGRGNYEELENSGDTTVESERPNKVTIP...,0000000000000000000000000000000000000000000000...,Membrane,207,TMEM154,['Nuclear membrane'],Approved,Nuclear_membrane,Q6P9G4
156,Q6P9G4,Homo sapiens (Human),Transmembrane protein 154,Non-AH,MQAPRAALVFALVIALVPVGRGNYEELENSGDTTVESERPNKVTIP...,0000000000000000000000000000000000000000000000...,Membrane,6049,TMEM154,['Nuclear membrane'],Approved,Nucleoplasm,Q6P9G4
176,Q86VR2,Homo sapiens (Human),Reticulophagy regulator 3,AH,MAEAEGVPTTPGPASGSTFRGRRDVSGSWERDQQVEAAQRALVEVL...,0000000000000000000000000000000000000000000000...,Membrane,169,RETREG3,"['Nucleoplasm', 'Nuclear membrane']",Approved,Nuclear_membrane,Q86VR2
177,Q86VR2,Homo sapiens (Human),Reticulophagy regulator 3,AH,MAEAEGVPTTPGPASGSTFRGRRDVSGSWERDQQVEAAQRALVEVL...,0000000000000000000000000000000000000000000000...,Membrane,4904,RETREG3,"['Nucleoplasm', 'Nuclear membrane']",Approved,Nucleoplasm,Q86VR2
